# Minimum Code for Training the Model with Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

# Enter the path to the src folder on your drive here.
%cd "drive/MyDrive/Module/05 Deep Learning/DeepLearning-FacialLandmarkDetection/src/"

In [ ]:
# If you're curious which GPU you're using :)
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

!nvidia-smi -L
!curl ipinfo.io

## Training

In [ ]:
from util.dataUtil import loadData
from util.modelUtil import saveModel
from modelStructure import defineModel
from training import trainModel

X_train, y_train, X_test = loadData(includeAugmented=True)
model = defineModel()

trainModel(
    model, X_train, y_train,
    epochs=100,
    batch_size=256,
    validation_split=0.2,
    showHistory=True
)

saveModel(model, "../models/modelV4")

## Testing

In [ ]:
from testing import testOnDataset, testOnVideoFile, testOnWebcam
from util.dataUtil import loadData
from util.modelUtil import loadModel

X_train, y_train, X_test = loadData()
numberOfPoints, seed = 20, 30

model = loadModel("../models/modelV4")
testOnDataset(model, X_train[seed:seed+numberOfPoints], trueValues=y_train[seed:seed+numberOfPoints], show=False, save=True, filename="V4.png")

## Evaluating

In [ ]:
from util.dataUtil import loadData, loadVideoTestData
from util.modelUtil import loadModel
from training import masked_accuracy, masked_mean_absolute_error, masked_mean_squared_error

# Loading the data
X_test, y_test = loadData(includeAugmented=True, percentageOfUncleanData=0)
X_videoTest, y_videoTest = loadVideoTestData()

# Loading the model
model = loadModel("../models/modelV4")

# recompile the model to use the custom metrics
model.compile(
    loss=masked_mean_squared_error,
    metrics=[masked_mean_absolute_error, masked_accuracy],
    run_eagerly=True
)

# Evaluating the model on the test data (batches of 32)
print(model.evaluate(x=X_test, y=y_test))
print(model.evaluate(x=X_videoTest, y=y_videoTest))